In [1]:
# imports necesarios
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier


### MNIST con K-vecinos más vercanos
En esta segunda sesión práctica vamos a entrenar clasificadores basados en distancias.


In [2]:
mnist = fetch_openml("mnist_784")

print(mnist.data.shape)
print(mnist.target.shape)

data = mnist.data
targets = mnist.target 

targets=targets.to_numpy()
targets=np.int8(targets)

data=data.to_numpy()
data=np.float32(data)



c:\Users\adri_\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


(70000, 784)
(70000,)


### Partición de los datos

Vamos a partir los datos en tres conjuntos: training, validation y test. Con un 80%, 10% y 10% respectivamente. 

Emplearemos el conjunto de training para aprender los parámetros del modelos, el conjunto de validation para escoger los mejores hiperparámetros. Finalmente reportaremos el resultado final sobre el conjunto de test.

### **Ejercicio 1**    

Realiza la partición de los datos en las particiones definidas (80%,10%,10%)

In [3]:
# Partición de los datos
X_train, X_aux, y_train, y_aux = train_test_split(data, targets, test_size=0.2, shuffle=True, random_state=23)
X_test, X_val, y_test, y_val = train_test_split(X_aux, y_aux, test_size=0.5, shuffle=True, random_state=23)

X_tv = np.concatenate((X_train, X_val), axis=0)
y_tv = np.concatenate((y_train, y_val), axis=0)

### **Ejercicio 2**   

Prueba un clasificador KNeighborsClassifier

In [4]:
## Prueba el clasificador KNeighborsClassifier
kn = KNeighborsClassifier()
kn.fit(X_train, y_train)
y_pred = kn.predict(X_val)
acc = accuracy_score(y_val, y_pred)
print(f'La precisión de {kn} es {acc:.2%}')

La precisión de KNeighborsClassifier() es 97.34%


### **Ejercicio 3**   

Encuentra el mejor valor de "K" con el conjunto de validación y obtén el resultados para test.

In [6]:
## Estimar el mejor parámetro K con el conjunto de validación
max = -1
best_k = 0

for k in [2,4,8,16,32,64]:
    kn = KNeighborsClassifier(n_neighbors=k)
    kn.fit(X_train, y_train)
    y_pred = kn.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    print(f'La precisión de {kn} es {acc:.2%}')

    if acc > max:
        max = acc
        best_k = k

print(f'Mejor n_neigbors={best_k}')

La precisión de KNeighborsClassifier(n_neighbors=2) es 96.91%
La precisión de KNeighborsClassifier(n_neighbors=4) es 97.40%
La precisión de KNeighborsClassifier(n_neighbors=8) es 97.13%
La precisión de KNeighborsClassifier(n_neighbors=16) es 96.67%
La precisión de KNeighborsClassifier(n_neighbors=32) es 95.96%
La precisión de KNeighborsClassifier(n_neighbors=64) es 95.14%
Mejor n_neigbors=4


In [7]:
max = -1
for k in range(best_k-2, best_k+3):
    kn = KNeighborsClassifier(n_neighbors=k)
    kn.fit(X_train, y_train)
    y_pred = kn.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    print(f'La precisión de {kn} es {acc:.2%}')

    if acc > max:
        max = acc
        best_k = k

print(f'Mejor n_neigbors={best_k}')

La precisión de KNeighborsClassifier(n_neighbors=2) es 96.91%
La precisión de KNeighborsClassifier(n_neighbors=3) es 97.36%
La precisión de KNeighborsClassifier(n_neighbors=4) es 97.40%
La precisión de KNeighborsClassifier() es 97.34%
La precisión de KNeighborsClassifier(n_neighbors=6) es 97.31%
Mejor n_neigbors=4


In [8]:
## Obtener tasa de acierto con el conjunto de test para el mejor K
kn = KNeighborsClassifier(n_neighbors=best_k)
kn.fit(X_tv, y_tv)
y_pred = kn.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f'La precisión de {kn} para el conjunto de test es {acc:.2%}')

La precisión de KNeighborsClassifier(n_neighbors=4) para el conjunto de test es 97.21%


## **Ejercicio 4**

Cambia la distancia que se emplea (por defecto la L2) por la L1. Ver la documentación para ver cómo poder definir funciones de distancia y pasarlas como parámetro. Esto ralentiza bastante la clasificación pero se puede emplear el parámetro "n_jobs" para paralelizar el proceso. Aún así el resultado es muy lento. Podríamos concluir que sk-learn no es una librería adecuada para clasificar con KNN con métricas definidas por el usuario. Por ejemplo no permite definir un algoritmo rápido de tipo kd_tree.

In [4]:
## Modifica la función de distancia

#  p = 1, this is equivalent to using manhattan_distance (l1)
for k in [1,2,3,4,5,6,7,8,9,10]:
    kn = KNeighborsClassifier(n_neighbors=k, n_jobs=-1, p=1)
    kn.fit(X_train, y_train)
    y_pred = kn.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    print(f'La precisión de {kn} para el conjunto de test es {acc:.2%}')

La precisión de KNeighborsClassifier(n_jobs=-1, n_neighbors=1, p=1) para el conjunto de test es 96.80%
La precisión de KNeighborsClassifier(n_jobs=-1, n_neighbors=2, p=1) para el conjunto de test es 95.93%
La precisión de KNeighborsClassifier(n_jobs=-1, n_neighbors=3, p=1) para el conjunto de test es 96.99%
La precisión de KNeighborsClassifier(n_jobs=-1, n_neighbors=4, p=1) para el conjunto de test es 96.57%
La precisión de KNeighborsClassifier(n_jobs=-1, p=1) para el conjunto de test es 96.79%
La precisión de KNeighborsClassifier(n_jobs=-1, n_neighbors=6, p=1) para el conjunto de test es 96.50%
La precisión de KNeighborsClassifier(n_jobs=-1, n_neighbors=7, p=1) para el conjunto de test es 96.59%
La precisión de KNeighborsClassifier(n_jobs=-1, n_neighbors=8, p=1) para el conjunto de test es 96.63%
La precisión de KNeighborsClassifier(n_jobs=-1, n_neighbors=9, p=1) para el conjunto de test es 96.56%
La precisión de KNeighborsClassifier(n_jobs=-1, n_neighbors=10, p=1) para el conjunto de

In [12]:
def my_dist(x, y):
    return np.sum(np.abs(x-y))

In [9]:
kn = KNeighborsClassifier(n_neighbors=4, n_jobs=4, metric=my_dist)
kn.fit(X_train, y_train)
y_pred = kn.predict(X_val[:100,:])
acc = accuracy_score(y_val[:100], y_pred[:100])
print(f'La precisión de {kn} para el conjunto de test es {acc:.2%}')

La precisión de KNeighborsClassifier(metric=<function my_dist at 0x00000229576F3060>, n_jobs=4,
                     n_neighbors=4) para el conjunto de test es 95.00%


In [15]:
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

max_acc = 0
best_k = 0
best_nc = 0

for nc in [2,4,8,16,32]:
    for k in [1,2,3,4,5,6,7,8,9,10]:
        kn = make_pipeline(
            PCA(n_components=nc),
            KNeighborsClassifier(n_neighbors=k, n_jobs=-1, p=1)
        ) 

        kn.fit(X_train, y_train)
        y_pred = kn.predict(X_val)
        acc = accuracy_score(y_val, y_pred)

        if acc > max_acc:
            max_acc = acc
            best_k = k
            best_nc = nc
print(f'La precisión MAX de {kn} para el conjunto de train/val es {max_acc:.2%}')

La precisión MAX de Pipeline(steps=[('pca', PCA(n_components=32)),
                ('kneighborsclassifier',
                 KNeighborsClassifier(n_jobs=-1, n_neighbors=10, p=1))]) para el conjunto de train/val es 97.51%


In [16]:
kn = make_pipeline(
            PCA(n_components=best_nc),
            KNeighborsClassifier(n_neighbors=best_k, n_jobs=-1, p=1)
        ) 

kn.fit(X_tv, y_tv)
y_pred = kn.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f'La precisión de {kn} para el conjunto de test es {acc:.2%}')

La precisión de Pipeline(steps=[('pca', PCA(n_components=32)),
                ('kneighborsclassifier',
                 KNeighborsClassifier(n_jobs=-1, n_neighbors=3, p=1))]) para el conjunto de test es 97.81%


In [21]:
from sklearn.ensemble  import VotingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

max_acc = -1
for nc in [2,4,8,16,32,64]:

    clf1 = HistGradientBoostingClassifier(max_bins=64, max_depth=16,min_samples_leaf=32)
    clf2 = HistGradientBoostingClassifier(max_bins=32, max_depth=16,min_samples_leaf=64)
    clf3 = HistGradientBoostingClassifier(random_state=22)
    clf4 = KNeighborsClassifier(n_neighbors=best_k, n_jobs=-1, p=1)
    t1 = DecisionTreeClassifier(max_depth=14, random_state=23)
    t2 = DecisionTreeClassifier(max_depth=14, random_state=24)
    t3 = DecisionTreeClassifier(max_depth=14, random_state=25)

    kn = make_pipeline(
                PCA(n_components=nc),
                VotingClassifier([
                    ('hg1',clf1),
                    ('hg2',clf2),
                    ('hg3',clf3),
                    ('kn', clf4),
                    ('t1',t1),
                    ('t2',t2),
                    ('t3',t3)
                ])
            ) 
    
    kn.fit(X_train, y_train)
    y_pred = kn.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    if acc > max_acc:
        max_acc = acc
        print(f'La precisión de {kn} para el conjunto de train/val es {acc:.2%}')
    
    

La precisión de Pipeline(steps=[('pca', PCA(n_components=2)),
                ('votingclassifier',
                 VotingClassifier(estimators=[('hg1',
                                               HistGradientBoostingClassifier(max_bins=64,
                                                                              max_depth=16,
                                                                              min_samples_leaf=32)),
                                              ('hg2',
                                               HistGradientBoostingClassifier(max_bins=32,
                                                                              max_depth=16,
                                                                              min_samples_leaf=64)),
                                              ('hg3',
                                               HistGradientBoostingClassifier(random_state=22)),
                                              ('kn',
                     